In [1]:
import numpy as np
import pandas as pd
import re

!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz, process

# Load data

In [6]:
all_data = pd.read_csv('all_data.csv')

# Filters for RSOs, which have type 'PUB', 'SAG', or 'SISG'.
rso_labels = ['PUB', 'SAG', 'SISG']
rso_data = all_data[all_data['Type'].isin(rso_labels)].reset_index(drop=True)

# Fix data types

The first issue to address is the formatting in the `Allocation` column - particularly for the string values "Sponsorship Only" and "Sponsorship only".

In [ ]:
# Filters for RSOs with sponsorships.
sponsored_rso_data = rso_data[(rso_data['Allocation'] == 'Sponsorship Only') |
                             (rso_data['Allocation'] == 'Sponsorship only')]

# Filters for RSOs allocations of $0.
unfunded_rso_data = rso_data[(rso_data['Allocation'] == '$0') |
                             (rso_data['Allocation'] == '$0.00')]

"Sponsorship Only" RSOs only appear in 2012, 2018, and 2019. Clubs allocated zero dollars only appear in 2013, 2014, 2015, 2016, and 2017, with the exception of four 2012 entries. Because the distributions of both categories are complementary, we can deduce that have similar meanings: While the clubs are still sponsored by the ASUC, they receive no funding from the ASUC. Thus, "Sponsorship Only" really means an allocation of $0.

In [63]:
# Replace "Sponsorship Only" RSOs with $0 of funding to make it consistent with the other allocation values.
rso_data.loc[:, 'Allocation'] = rso_data.loc[:, 'Allocation'].replace(['Sponsorship only', 'Sponsorship Only'], '$0')

In [64]:
# Remove '$' and ',' characters to convert the strings to float value.
rso_data.loc[:, 'Allocation'] = rso_data.loc[:, 'Allocation'].str.replace('$', '')
rso_data.loc[:, 'Allocation'] = rso_data.loc[:, 'Allocation'].str.replace(',', '')

# The STARR Referendum is mentioned in the 2018-2019 allocations; we'll save the names of these groups for reference.
rso_STARR = ['bridges Multicultural Resource Center',
            'Indigenous and Native Coalition Recruitment and Retention Center',
            'Middle Eastern North African Recruitment and Retention Center',
            'Mixed Student Union',
            'Pilipinx Academic Student Services',
            'RaÌ_ces Recruitment and Retention Center (formerly Raza Recruitment and Retention Center)',
            'REACH! Asian Pacific American Recruitment and Retention Center']

# The following line confirms this information.
rso_data[(rso_data['Year'] == '2018') & (rso_data['Organization'].isin(rso_STARR))]

# Now back to cleaning the 'Allocations' column.
def clean_allocation(alloc):
    result = re.search(r'[0-9]+[.]*[0-9]*', alloc)
    if result:
        return result[0]

rso_data.loc[:, 'Allocation'] = rso_data.loc[:, 'Allocation'].apply(clean_allocation)
rso_data.loc[:, 'Allocation'] = rso_data.loc[:, 'Allocation'].astype(float)

In [ ]:
# Convert 'Standing' from float to integer.
rso_data[['Standing']] = rso_data[['Standing']].astype(int)

# Clean CalLink designation data

In order to ensure that RSO names are consistent across budget years, each RSO can be matched to their CalLink names. We'll start by cleaning the CalLink data, `callink_data.txt`.

In [149]:
desig_data = pd.read_csv('data/callink_data.txt')[['Organization', 'Designation']]

In [ ]:
# When clubs don't add a profile picture to their CalLink page, they instead have a
# circle with the first letter of their org. The following code will filter this out of
# the webscraped names.
def ws_name_cleaner(name):
    return re.search(r'.\n(.*)', name)[1]

n_idx = desig_data[desig_data['Organization'].str.contains('\n')].index
desig_data.loc[n_idx, 'Organization'] = desig_data.loc[n_idx, 'Organization'].apply(ws_name_cleaner)

In [ ]:
# Some campus departments also have asterisks around their organization names; filter those out.
def ws_asterisk_cleaner(name):
    result = re.search(r'\*(.*)\*', name)
    if result != None:
        return result[1]
    return name

desig_data['Organization'] = desig_data['Organization'].apply(ws_asterisk_cleaner)

In [ ]:
# Now let's clean the designations. Each one has an extra space after the name, so that'll
# be an easy fix.
def del_space(name):
    if name[-1] == ' ':
        return name[:-1]
    return name

desig_data['Designation'] = desig_data['Designation'].apply(del_space)

In [ ]:
# We can now look at all the designations.
desig_count = desig_data.groupby('Designation').count().reset_index()
desig_count

In [ ]:
# A list of designations with more than 10 organizations is shown below. 
valid_desigs = desig_count[desig_count['Organization'] >= 10]['Designation']
valid_desigs

In [ ]:
# We can extract the typical RSO categories from them to create a list of "regular designations" for RSOs.
# We'll ignore the other designations for now.
reg_desigs = [i for i in valid_desigs if 'RSO' in i]
reg_desigs

In [ ]:
# To account for outliers / inconsistent designations, we'll make a list of all the designations 
# that have less than 10 organizations under that category for closer examination.
odd_desigs = desig_count[desig_count['Organization'] < 10]
odd_desigs 

# There are definitely naming overlaps! Time to fix those.

In [ ]:
# Editing the academic RSOs in 'odd_desigs'.
academic = odd_desigs[odd_desigs['Designation'].str.contains('Academic')]['Designation']
desig_data[desig_data['Designation'].isin(academic)]

# While Der Kreis and Energy and Resources Grad Students seem academic, iGEM appears to be
# more tech / project oriented. These orgs will be edited accordingly.
desig_data.at[[498, 534], 'Designation'] = 'Academic RSO'
desig_data.at[750, 'Designation'] = 'Technology RSO'

# Editing the cultural RSOs.
cultural = odd_desigs[odd_desigs['Designation'].str.contains('Cultural')]['Designation']
desig_data[desig_data['Designation'].isin(cultural)]

desig_data.at[745, 'Designation'] = 'Cultural & Identity/LGBTQ+ RSO'
desig_data.at[[1384, 1387], 'Designation'] = 'Cultural & Identity/Women Interest RSO'

# Environmental RSO.
desig_data[desig_data['Designation'] == 'Environmental']
desig_data.at[256, 'Designation'] = 'Environmental & Sustainability RSO'

# Professional RSO.
desig_data[desig_data['Designation'].str.contains('Professional RSO - ')]
desig_data.at[427, 'Designation'] = 'Professional RSO'

# Media RSO.
desig_data[desig_data['Designation'] == 'Media RSO']
desig_data.at[723, 'Designation'] = 'Media & Film RSO'

# Performing arts RSO.
desig_data[desig_data['Designation'] == 'Performing Arts']
desig_data.at[297, 'Designation'] = 'Performing Arts RSO'

# Political RSO.
desig_data[(desig_data['Designation'].str.contains('Political'))
          & (desig_data['Designation'] != 'Political & Advocacy RSO')]
desig_data.at[[58, 1138], 'Designation'] = 'Political & Advocacy RSO'

# Service RSO.
desig_data[(desig_data['Designation'].str.contains('Service'))
          & (desig_data['Designation'] != 'Service RSO')]
desig_data.at[[359, 449, 505, 520, 881, 961], 'Designation'] = 'Service RSO'

# Spiritual RSO.
desig_data[(desig_data['Designation'].str.contains('Spiritual'))
          & (desig_data['Designation'] != 'Spiritual RSO')]
desig_data.at[[201, 335, 789], 'Designation'] = 'Spiritual RSO'

# Sponsored RSO.
desig_data[desig_data['Designation'].str.contains('Sponsored')]
desig_data.at[[280, 306, 754, 904, 912, 1037, 1100, 1102], 'Designation'] = 'Sponsored RSO'

# It doesn't seem like pages with no real designation are relevant to RSOs, but
# let's unify the label.
blank = ['Inactive Page', 'No Response']
desig_data[desig_data['Designation'].isin(blank)]
desig_data.at[[100, 642, 964], 'Designation'] = 'None'

# Finally, we can extract RSO designations for future use. 
rso_desigs = desig_data[desig_data['Designation'].str.contains('RSO')]

# Merge designations with `rso_data`

In [ ]:
rso_data = pd.merge(rso_data, rso_desigs, how='left', on='Organization')

rso_data['Designation'] = rso_data['Designation'].fillna('None')

# Rearranges column order.
cols = ['Year', 'Organization', 'Type', 'Designation', 'Standing', 'Allocation']
rso_data = rso_data[cols]

len(rso_data[rso_data['Designation'] == 'None'].index)

# Edit abbreviated RSO names

In [ ]:
# Here, we'll manually address and edit the names of particular organizations with abbreviations.
# We generally want these edits to be consistent with 1) the group's most updated name, and
# 2) the group's name on CalLink, the official hub for Berkeley clubs (although this excludes
# former names of the club).

# Berkeley Opinion seems that it doesn't usually go by BerkOp - defaulting to the full name.
rso_data.loc[[827, 2176], 'Organization'] = 'Berkeley Opinion'
rso_data[rso_data['Organization'].str.contains('Berkeley Opinion')]


# OASES, while commonly known by the abbreviation, formally lists its organization under the
# full name, 'Oakland Asian Students Education Services'. We'll leave it as is.
rso_data[rso_data['Organization'].str.contains('Oakland Asian')]


# Movimiento Estudiantil Chicano/xicana de Atzlan doesn't tyically go by its abberviation, MEChxA.
rso_data.loc[[287,692], 'Organization'] = 'Movimiento Estudiantil Chicano/xicana de Atzlan'
rso_data[rso_data['Organization'].str.contains('Estudiantil')]

# Project SMILE has a pretty long official name.
rso_data.at[[298, 708, 1127, 1567, 2057, 2533], 'Organization'] = 'Project Spreading Multiculturalism and Inspiring Leadership through Education'
rso_data[rso_data['Organization'].str.contains('Project Spreading')]

# Chicano(a)s/Latino(a)s in Health Education has some odd mistypes
rso_data.loc[[261, 653, 1068, 2465], 'Organization'] = 'Chicano(a)s/Latino(a)s in Health Education'
rso_data[rso_data['Organization'].str.contains('Health Education')]

# Consult Your Community (CYC) at Berkeley
rso_data.at[456, 'Organization'] = 'Consult Your Community (CYC) at Berkeley'
rso_data[rso_data['Organization'].str.contains('CYC')]

# FEM Tech
rso_data.at[[1761, 2715], 'Organization'] = 'FEMTech'
rso_data[rso_data['Organization'].str.contains('FEM')]

# EGO
rso_data.at[[110, 476, 882, 1291, 1750], 'Organization'] = 'EGO (Cal\'s Traditional Korean Percussion Group)'
rso_data[rso_data['Organization'].str.contains('EGO')]

# iHeart
rso_data[rso_data['Organization'].str.contains('iHeart')]

# DULCE
rso_data.at[[660, 2480], 'Organization'] = 'DULCE (Diabetes: Unidos Lograremos Controlar Esta Enfermedad)'
rso_data[rso_data['Organization'].str.contains('DULCE')]

# Cal Literature and Arts Magazine
rso_data.at[[10, 762, 1181, 1623, 2112, 3022], 'Organization'] = 'Cal Literature & Arts Magazine'
rso_data[rso_data['Organization'].str.contains('Cal Literature')]

# Institute of Transportation Engineers at Berkeley
rso_data.at[[141, 429], 'Organization'] = 'Institute of Transportation Engineers at Berkeley'
rso_data[rso_data['Organization'].str.contains('Transportation Engineers')]

# Sigma Iota Rho
rso_data.at[1409, 'Organization'] = 'Sigma Iota Rho: International & Area Studies Honor Society'
rso_data[rso_data['Organization'].str.contains('Sigma Iota')]

# Fixes weird character errors for Raíces
rso_data.loc[3394, 'Organization'] = 'Raíces Recruitment and Retention Center'

# Match RSOs without a designation

The following functions use the `fuzzywuzzy` package to match RSOs without a designation to similarly named organization in `rso_desigs`. If the match is authenticated by the user, then the RSO name is replaced with its CalLink counterpart and assigned its designation.

In [ ]:
# Helper functions for the main function, desig_assigner.

# Sifts through the possible designations before manually selecting a match. All decisions must be approved by the user.
def desig_selector(potential_desigs):
    for i in potential_desigs:
        print(i[0])
        auth = input('y/n?: ')

        if auth == 'y':
            return i
            
# Cleans and relabels names according to CalLink, as well as assigns designations.
def clean_name_desigs(rso_data, rso_desigs, org_idx, desig):
    org_name = rso_data['Organization'][org_idx]
    
    callink_name = desig[0]
    desig_idx = desig[2]
    designation = rso_desigs.at[desig_idx, 'Designation']
    
    if callink_name != org_name:
        rso_data.at[org_idx, 'Organization'] = callink_name
                    
        org_idxs = rso_data[rso_data['Organization'] == callink_name].index               
        rso_data.at[org_idxs, 'Designation'] = designation

In [151]:
# Main function for assigning designations.
def desig_assigner(rso_data, rso_desigs):
    data = rso_data[(rso_data['Designation'] == 'None')]
    manual_fixes = []
    
    for i in data.index:
        org = data['Organization'][i]
        
        print('Organization Name: ' + org + '\n' + '----------')
        print('Desig check:')
        
        potential_desig = process.extract(org, rso_desigs['Organization'], limit=2)
        desig = desig_selector(potential_desig)

        if desig != None:
            clean_name_desigs(rso_data, rso_desigs, i, desig)
            
        print('---------')

In [153]:
desig_assigner(rso_data, rso_desigs)

Organization Name: 0 Comedy
----------
Desig check:
Bootleg Comedy (formerly Mainline Comedy Force)
y/n?: 
TBD Comedy
y/n?: 
---------
Organization Name: 3DMC (under Engineering Student Council)
----------
Desig check:
Engineering Student Council
y/n?: 
Afghan Student Association
y/n?: 
---------
Organization Name: Berkeley Formula Racing (under Engineering Student Council)
----------
Desig check:
Engineering Student Council
y/n?: 
180 Degrees Consulting at Berkeley
y/n?: 
---------
Organization Name: Berkeley Hyperloop (under Engineering Student Council)
----------
Desig check:
Engineering Student Council
y/n?: 
180 Degrees Consulting at Berkeley
y/n?: 
---------
Organization Name: Berkeley Opinion
----------
Desig check:
180 Degrees Consulting at Berkeley
y/n?: 
3D Modeling and Animation at Berkeley
y/n?: 
---------
Organization Name: Black Campus Ministries (under InterVarsity Christian Fellowship)
----------
Desig check:
InterVarsity Christian Fellowship
y/n?: 
Adventist Christian 

y/n?: y
---------
Organization Name: Project pengyou, Berkeley Chapter
----------
Desig check:
Project Pengyou, Berkeley Chapter
y/n?: y
---------
Organization Name: QJew (under QARC)
----------
Desig check:
QJew
y/n?: y
---------
Organization Name: Queer + Allied Pre-Health/ Pre-Medical Association
----------
Desig check:
Queer + Allied Pre-Health / Pre-Medical Association
y/n?: y
---------
Organization Name: Queer Trans South Asians at Berkeley (under QARC)
----------
Desig check:
7 Cups at Berkeley
y/n?: 
A Better Way at Berkeley
y/n?: 
---------
Organization Name: Re-Entry and Transfer Student Association
----------
Desig check:
Re-entry and Transfer Student Association
y/n?: y
---------
Organization Name: Robobears (under ESC)
----------
Desig check:
RoboBears
y/n?: y
---------
Organization Name: Robotics & Engineering for Youth (under ESC)
----------
Desig check:
Robotics & Engineering for Youth
y/n?: y
---------
Organization Name: Robotics at Berkeley  (under Engineering Student

Engineering Student Council
y/n?: 
3D Modeling and Animation at Berkeley
y/n?: 
---------
Organization Name: Financial Literacy and Economic Justice Conference (under CP)
----------
Desig check:
Financial Literacy and Economic Justice Conference
y/n?: y
---------
Organization Name: INvent Berkeley
----------
Desig check:
INvent
y/n?: y
---------
Organization Name: Kidney Disease Screening and AwarenESC Program
----------
Desig check:
Kidney Disease Screening and Awareness Program
y/n?: y
---------
Organization Name: Lambda Theta Nu Sorority, Inc.
----------
Desig check:
Eta Kappa Nu
y/n?: 
Future Business Leaders of America-Phi Beta Lambda, Inc.
y/n?: 
---------
Organization Name: Lu-mien Student Union
----------
Desig check:
Iu-Mien Student Union
y/n?: y
---------
Organization Name: Oakland Asian Student Educational Services (OASES) (under CP)
----------
Desig check:
Oakland Asian Student Educational Services
y/n?: y
---------
Organization Name: People's Test Preparation Service (unde

['0 Comedy',
 '3DMC (under Engineering Student Council)',
 'Berkeley Formula Racing (under Engineering Student Council)',
 'Berkeley Hyperloop (under Engineering Student Council)',
 'Berkeley Opinion',
 'Black Campus Ministries (under InterVarsity Christian Fellowship)',
 'BusinESC Transfer Network',
 'Cal Ace Space ',
 'Cal Seismic Desgin Team (under Engineering Student Council)',
 'Cal Super Mileage Vehicle (CalSMV) (under ESC)',
 'Cal Transportation Team (under Engineering Student Council)',
 'Chec Consulting (under Engineering Student Council)',
 'ChemE Car (under Engineering Student Council)',
 'Chi Episilon Civil Engineering Honor Society (under ESC) ',
 'De Colores (under QARC)',
 'Deewani',
 'Delta Xi Phi Multicultural Sorority Inc.',
 'Design Engineering Collaborative (under Engineering Student Council)',
 'Engineering 4 Kids (under Engineering Student Council)',
 'Engineering Solutions @ Berkeley (under Engineering Student Council)',
 'Faces of African Muslims (under Muslim S

# Assign designations manually (section to be cleaned)

Unfortunately, many designations are still missing in `rso_data`. They'll have to be manually inputted based on extensively searching CalLink.

In [269]:
# 'Female Empowerment and Mentoring in Technology', 'Berkeley Formula Racing', '0 Comedy', 'smART'

# rso_data_2020[rso_data_2020['Organization'].str.contains('Women and Youth')]

# rso_data_2020.at[27, 'Organization'] = '3DMC'
# rso_data_2020.at[299, 'Designation'] = 'Arts RSO'

# test = pd.concat([rso_data, rso_data_2020])
# test[test['Organization'].str.contains('0')]

# rso_data = pd.concat([rso_data, rso_data_2020]).reset_index(drop=True)
# rso_data[rso_data['Organization'].str.contains('Hyperloop')]
rso_data_2020

,Year,Organization,Type,Designation,Standing,Allocation
0,2020,BARE Magazine,PUB,Publications RSO,13,3262.0
1,2020,Berkeley Fiction Review,PUB,Publications RSO,37,2200.0
2,2020,Berkeley Poetry Review,PUB,Publications RSO,26,1216.0
3,2020,Berkeley Science Review,PUB,Publications RSO,20,1200.0
4,2020,Berkeley Scientific Journal,PUB,Publications RSO,4,900.0
5,2020,Blue and Gold Yearbook,PUB,Publications RSO,6,560.0
6,2020,Cal Literature & Arts Magazine,PUB,Publications RSO,29,3000.0
7,2020,Caliber Magazine,PUB,Publications RSO,11,9500.0
8,2020,Caravan Travel & Style Magazine,PUB,Publications RSO,3,250.0
9,2020,Clio's Scroll,PUB,Publications RSO,21,1400.0


In [283]:
# Import our modified rso_data.

rso_data = pd.read_csv('rso_data.txt')[['Year', 'Organization', 'Type', 'Designation', 'Standing', 'Allocation']]

In [48]:
not_rso = ['Sigma Pi Alpha', 'Public Service Internships', 'Omega Phi Beta', 'Multi-Cultural Greek Council',
           'Lambda Theta Phi', 'Lambda Theta Nu', 'Lambda Theta Alpha', 'Delta Xi Phi', 'Gamma Rho Lambda',
           'Gamma Zeta Alpha', 'Greening the Greeks', 'Delta Sigma Theta', 'Alpha Phi Alpha', 'Alpha Kappa Alpha', 
           'Sigma Epsilon Omega', 'Sigma Omicron Pi', 'Sigma Psi Zeta']

re_not_rso = 'Sigma Pi Alpha|Public Service Internships|Omega Phi Beta|Multi-Cultural Greek Council|Lambda Theta Phi|Lambda Theta Nu|Lambda Theta Alpha|Delta Xi Phi|[G|g]amma [R|r]ho [L|l]ambda|Gamma Zeta Alpha|Greening [t|T]he Greeks|Delta Sigma Theta|Alpha Phi Alpha|Alpha Kappa Alpha|Sigma Epsilon Omega|Sigma Omicron Pi|Sigma Psi Zeta|Sigma Gamma Rho|UC Berkeley Public Service Center|National Pan-Hellenic|Democratic Education|CalGreeks|CalGreek|Lambda Upsilon Lambda|Phi Beta Sigma|Phi Sigma Rho|Order of Omega|CalTV|Open Computing Facility|Iota Phi Theta|Cal Corps|Sigma Omicron Pi|Alpha Kappa Lambda|Alpha Sigma Nu'

rso_data[(rso_data['Designation'] == 'None') & (~rso_data['Organization'].str.contains(re_not_rso))]
# rso_data[(rso_data['Standing'] == 15) & (rso_data['Year'] == 2017)]


,Year,Organization,Type,Designation,Standing,Allocation
3341,2019,Sigma Omicron Pi,SAG,None,1,0.0


In [ ]:
# The following RSOs were unsearchable and were assigned designations based on educated inferences.
#'Golden Paw Productions', 'California Patriot', 'Take Back the Night', 'Save a Refugee',
# 'Disabled Students' Union', 'American Parliamentary Debate Society of Berkeley', 'Raza Caucus', 'Project Nutrition'
# 'Berkeley City College Service Community', 'Students for Liberty', 'Delta Sigma Pi', 'Stop the Traffick',
# 'Students for Integrative Medicine', 'Students Against Fracking', 'Malaria No More', 'League of Legends at Berkeley',
# 'Health in All Policies', 'Cal Set Design', 'Black Political Law Association',
# 'Camp WOW (Week of Wilderness)', 'Cal Facilitation Team', 'Understanding Physics', 'UC Berkeley Men's Ice Hockey',
# 'Student Human Resources Association', 'Student Commuters at Cal', 'Speech and Debate at Berkeley', 
# 'ProActive Wellness', 'American Marketing Association', 'Korean Student Organizations', 'Investment Banking Club',
# 'Eritrean Student Association', 'Chinese Instrumental Society', 'Blues at Berkeley', 'Bengali Students Association',
# 'Cal Health Coalition', 'Human Journal at Berkeley', 'Kappa Alpha Delta', 'Fitness at Berkeley',
# 'Mechanism of Feeling', 'Big Ideas@Berkeley', 'United Pre-Health Peers', 'Escape the Lights Social Club',
# 'Career Attainment in Responsible Enterprises', 'Engineering Innovation & Consulting', 'Berkeley Energy',
# 'Alumni of Ecole Polytechnique at Berkeley', 'Cal Scholars Chapter', 'Freshman Sophomore Entrepreneurial Society'
# 'Mak', 'International Student Fellowship', 'Multi-Ethnic Student House'

rso_data[(rso_data['Organization'].str.contains('International Student'))]
# idx = rso_data[(rso_data['Organization'].str.contains('Latinx Architecture'))].index

rso_data.at[1693, 'Organization'] = 'Berkeley Hyperloop'
# rso_data.at[502, 'Designation'] = 'Cultural & Identity/International RSO'


# rso_data[(rso_data['Designation'].str.contains('LGBTQ'))]

# rso_data.iloc[652]
# rso_data['Designation'].unique()

In [287]:
old_rso_data[old_rso_data['Organization'].str.contains('Ethiopian')]
# old_rso_data.iloc[876]

,Year,Organization,Type,Standing,Allocation
115,2012,EthiopianStudentUnion,SAG,17,"$1,680.00"
483,2013,Ethiopian Student Union,SAG,18,"$1,000"
898,2014,Horn of Africa Student Association (formerly Ethiopian Student Union),SAG,19,"$1,000.00"


In [310]:
rso_data.to_csv('rso_data.txt')

In [78]:
# 1938, 1939; 'Volunteers Around the World'; 'Cal Bhangra' 1772, 1773; 'Berkeley Energy' 1679; 1816, 1817; 
# 'Women in Economics' 1938, 1939; 'Fitness at Berkeley'; 'Dreamers Project Mentorship Program' 2478, 2479;
# 'Surf Team' 1911, 1912; 'Foodino'; 'Model United Nations' 2172, 2410; BareAbundance/Feeding Forward; 
# 'Korean- American Scientists' 1810, 1811; 3341


rso_data = rso_data[~rso_data['Organization'].str.contains(re_not_rso)]
# rso_data = rso_data.drop([1938, 1686, 1773, 1816, 2478, 1911, 1810, 628])
rso_data[rso_data['Designation'] == 'None']
# rso_data.loc[3341]
# rso_data.at[2172, 'Organization'] = 'Berkeley Model United Nations Conference'
# rso_data[rso_data['Organization'].str.contains('Feeding F')]

,Year,Organization,Type,Designation,Standing,Allocation


In [ ]:
name_changes = rso_data[rso_data['Organization'].str.contains('former|Former|formely', regex=True)]
name_idx = name_changes.index

In [ ]:
name_dict = {'Collegiate StarLeague at Berkeley': 'eSports at Berkeley', 
               'Asian American Health Society': 'Pre-Health Society',
               'International Association of Business Communicators': 'Berkeley Business Society',
               'Conscious Living Collective': 'Conscious Network',
               'Ethiopian Student Union': 'Horn of Africa Student Association',
               'Global Medical Brigades': 'Global Health Brigades',
               'Project Nutrition': 'Picknic',
               'Square Up': 'Golden Squares',
               'Italian International Student Association': 'Italian Society at Berkeley',
               'Al-Bayan': 'threads',
               'STOP the Traffick': 'Anti-Trafficking Coalition at Berkeley',
               'Stop the Traffick': 'Anti-Trafficking Coalition at Berkeley',
               'SIFE': 'Enactus of Berkeley',
               'Autism Speaks U at Berkeley': 'Spectrum: Autism at Cal',
               'Creative Marketing Group': 'CMG Strategy Consulting',
               'Californium Brewing and Winemaking': 'Ferments at Berkeley',
               'Juggers of the Seven Regents': 'Jugger at Berkeley',
               'Latino Pre-Law Society': 'Latinx Pre-Law Society',
               'Undergraduate Statistics Association': 'Statistics Undergraduate Students Association',
               'Undergraduate Statistics Association at Berkeley': 'Statistics Undergraduate Students Association',
               'Berkeley Disaster Response Team': 'Berkeley Disaster Team',
               'Raza Recruitment and Retention Center': 'Raíces Recruitment and Retention Center',
               'Raza Recruitment & Retention Center': 'Raíces Recruitment and Retention Center',
               'True Asian Leaders': 'Thrive Aspire Lead',
               'Krása Cosmetics Team': 'Aurum Cosmetics',
               '100 College Black Men, Berkeley Campus': 'Collegiate 100',
               'EPOCH': 'Product Development at Berkeley'
              }

In [14]:
# Jugger at Berkeley (@ idx 2751) has is missing parentheses and is fixed below.

rso_data.at[2751, 'Organization'] = 'Jugger at Berkeley (formerly known as Juggers of the Seven Regents)'

In [16]:
# Some test cases for clean_name.

a = 'asian american [formerly balh blah]'
b = 'threads (formerly known as Al-Bayan)'
c = 'Spectrum: Autism at Cal (Formerly known as Autism Speaks U at Berkeley) (Spectrum)'
e = 'Jugger at Berkeley formerly known as Juggers of the Seven Regents'
f = 'Enactus of Berkeley (formely SIFE)'
g = 'Engineering Student Council (formerly Engineer\'s Joint Council)'
h = 'Berkeley Business Society (formerly International Association of Business Comm'

clean_name(f)

'Enactus of Berkeley'

In [17]:
# Filter out former naming tags.

rso_data.loc[name_idx, 'Organization'] = rso_data.loc[name_idx, 'Organization'].apply(clean_name)

In [18]:
# Apply name changes with name_dict.

def change_name(name):
    if name in name_dict:
        return name_dict[name]
    return name

rso_data.loc[:, 'Organization'] = rso_data.loc[:, 'Organization'].apply(change_name)

In [19]:
# Many RSOs also have abbreviations or alternative names.

shorthand = rso_data[rso_data['Organization'].str.contains('\(.*\)', regex=True)]

# It seems like there are inconsistencies with whether the groups go by the abbreviation or 
# the full name of the group. So we'll just have to cross-reference how the groups brand themselves
# over time and go with the most frequent use.

,Year,Organization,Type,Designation,Standing,Allocation
10,2012,CLAM (Cal Literature and Arts Magazine),PUB,None,15,1495.00
30,2012,To An Unknown God (TAUG),PUB,None,4,1870.00
33,2012,acts2fellowship (A2F),SAG,None,11,2807.80
36,2012,Alpha Kappa Psi (AKPsi),SAG,None,2,52.50
42,2012,American Society of Civil Engineers (ASCE),SAG,None,8,100.00
67,2012,Berkeley Undergraduate Sociology Association (BUSA),SAG,None,5,105.00
74,2012,Cal Berkeley Democrats (CBD),SAG,None,15,4020.47
84,2012,Cal Undergraduate Public Health Coalition (UPHC),SAG,None,9,1419.60
94,2012,College Ski and Snowboard Club (CSSC),SAG,None,14,210.00
109,2012,Eggster Organization (Eggster??s Committee),SAG,None,12,1730.45


In [22]:
# # It seems that there are currently 1746 entries in the data that have no designation.
# # Time to fix that. We'll start by year and go backwards.

# unsure = rso_data.at[[3026, 3028], 'Organization']

# rso_data.at[3020, 'Organization'] = 'BerkTown Buzz'
# rso_data.at[[759, 3021], 'Organization'] = 'Blue and Gold Yearbook'
# rso_data.at[3027, 'Organization'] = 'Intercollegiate Finance Journal at Berkeley'


# # rso_data[rso_data['Organization'].str.contains('Latin')]
# # rso_desigs[rso_desigs['Organization'].str.contains('US')]

rso_data[(rso_data['Designation'] == 'None') & (rso_data['Year'] == '2019')]

,Year,Organization,Type,Designation,Standing,Allocation
3020,2019,Berktown Buzz,PUB,None,1,700.0
3021,2019,Blue & Gold Yearbook,PUB,None,5,700.0
3022,2019,Cal Literature & Arts Magazine,PUB,None,28,2800.0
3026,2019,In Color Film and Visual Arts Collective,PUB,None,1,400.0
3027,2019,Intercollegiate Finance Journal at UC Berkeley,PUB,None,1,700.0
3028,2019,LatinUS,PUB,None,1,500.0
3029,2019,Maganda Magazine,PUB,None,30,610.0
3038,2019,The Heuristic Squelch,PUB,None,23,7999.0
3039,2019,threads,PUB,None,20,11000.0
3045,2019,A.S.T.R.O. - Astronomer's Society for Teaching Recreation and Outreach,SAG,None,3,0.0


In [108]:
# What about groups that absorb smaller groups?

anti_trafficking = ['Anti-Trafficking Coalition at Berkeley',
                    'Student Abolitionist Movement',
                    'International Justice Mission',
                    'International Justice Mission Campus Chapter at Berkeley'
                   ]

bridges = []

sustainable_housing_at_california = ['Tiny House in My Backyard', 'Solar Decathlon']
# Or groups that split up?

speech_and_debate = ['Speech at Berkeley', 'Parliamentary Debate at Berkeley']

# Groups that are now funded by the ASUC(?)

decal = rso_data[rso_data['Organization'].str.contains('DeCal')]

Somali, Ethiopian, Eritrean, South Sudanese, Sudanese Association (SEE§A) (formerly as Horn of Africa Student Association - HASA)

In [ ]:
Phi Gamma Nu
KASA

In [304]:
# Removes 'formerly' tags from RSO names.

def clean_name(name):
    return re.search(r'(.*) [[(][Ff]or', name)[1]

def clean_the(name):
    return 'The ' + re.search(r'(.*), The', name)[1]

In [309]:
# formers_idx = rso_data[rso_data['Organization'].str.contains('[\(\[][Ff]or', regex=True)].index
# rso_data.loc[formers_idx, 'Organization'] = rso_data.loc[formers_idx, 'Organization'].apply(clean_name)

# the_idx = rso_data[(rso_data['Organization'].str.contains(', The')) & 
#          (~rso_data['Organization'].str.contains('Rishi|Therapists'))].index

# rso_data.loc[the_idx, 'Organization'] = rso_data.loc[the_idx, 'Organization'].apply(clean_the)
rso_data

,Year,Organization,Type,Designation,Standing,Allocation
0,2012,maganda magazine,PUB,Publications RSO,23,1592.02
1,2012,Threads,PUB,Publications RSO,12,1300.00
2,2012,Alternative Breaks,PUB,Service RSO,10,891.07
3,2012,BARE Magazine,PUB,Publications RSO,6,1250.00
4,2012,Berkeley Fiction Review,PUB,Publications RSO,29,1036.99
5,2012,Berkeley Poetry Review,PUB,Publications RSO,18,1500.00
6,2012,Berkeley Review of Education,PUB,Publications RSO,1,500.00
7,2012,Berkeley Science Review,PUB,Publications RSO,13,2139.47
8,2012,Caliber Magazine,PUB,Publications RSO,3,1700.00
9,2012,Cal Literature & Arts Magazine,PUB,Publications RSO,15,1495.00


In [ ]:
consistent = rso_data.groupby(['Organization']).count()
consistent = consistent[consistent['Year'] == 4].sort_values('Year')
#consistent.index

#rso_data[(rso_data['Organization'].isin(consistent.index)) & (rso_data['Standing'] == 1)]
rso_data[(rso_data['Standing'] == 0) & (rso_data['Allocation'] != 0)]
rso_data[rso_data['Organization'] == 'Anthropology Undergraduate Association']

#rso_data[(rso_data['Standing'] == 1) & (rso_data['Year'] == '2016')]
#rso_data[rso_data['Organization'] == 'Cal Queer & Asian']

zero_standing = rso_data[rso_data['Standing'] == 0]['Organization']
zero_std_overall = rso_data[rso_data['Organization'].isin(zero_standing)].sort_values(['Organization', 'Year'])
zero_std_overall

2016 contains clubs with 0 standing... these are presumably new clubs. However, this is inconsistent with other years, where it seems that new clubs begin with a standing of 1. 

Example: Quidditch League at Berkeley, self-proclaimed to have been founded in Fall 2009 on their Facebook page, https://www.facebook.com/calquidditch/. This makes sense when you line up the years from 2017 and onward. The strange standings is probably due the the club going inactive...

Example: Anthropology Undergraduate Association seems to be on and off? Probably, again, due to club inactivity.

Example: Cal Literature & Arts Magazine is whack.